<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "George", "transactions": [{"transaction-id": 22, "amount": 2007}, {"transaction-id": 488, "amount": 1853}, {"transaction-id": 507, "amount": 1890}, {"transaction-id": 508, "amount": 1677}, {"transaction-id": 933, "amount": 1704}, {"transaction-id": 979, "amount": 1901}, {"transaction-id": 1048, "amount": 1792}, {"transaction-id": 1193, "amount": 1757}, {"transaction-id": 1619, "amount": 1613}, {"transaction-id": 1646, "amount": 1656}, {"transaction-id": 1861, "amount": 1821}, {"transaction-id": 2000, "amount": 1529}, {"transaction-id": 2036, "amount": 1754}, {"transaction-id": 2289, "amount": 1379}, {"transaction-id": 2319, "amount": 1642}, {"transaction-id": 2678, "amount": 1854}, {"transaction-id": 3017, "amount": 1763}, {"transaction-id": 3101, "amount": 1899}, {"transaction-id": 3114, "amount": 1866}, {"transaction-id": 3234, "amount": 1870}, {"transaction-id": 3286, "amount": 1724}, {"transaction-id": 3454, "amount": 1903}, {"transaction-id": 3514, "amount": 1

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "George", "transactions": [{"transaction-id": 22, "amount": 2007}, {"transaction-id": 488, "amount": 1853}, {"transaction-id": 507, "amount": 1890}, {"transaction-id": 508, "amount": 1677}, {"transaction-id": 933, "amount": 1704}, {"transaction-id": 979, "amount": 1901}, {"transaction-id": 1048, "amount": 1792}, {"transaction-id": 1193, "amount": 1757}, {"transaction-id": 1619, "amount": 1613}, {"transaction-id": 1646, "amount": 1656}, {"transaction-id": 1861, "amount": 1821}, {"transaction-id": 2000, "amount": 1529}, {"transaction-id": 2036, "amount": 1754}, {"transaction-id": 2289, "amount": 1379}, {"transaction-id": 2319, "amount": 1642}, {"transaction-id": 2678, "amount": 1854}, {"transaction-id": 3017, "amount": 1763}, {"transaction-id": 3101, "amount": 1899}, {"transaction-id": 3114, "amount": 1866}, {"transaction-id": 3234, "amount": 1870}, {"transaction-id": 3286, "amount": 1724}, {"transaction-id": 3454, "amount": 1903}, {"transaction-id": 3514, "amount": 1

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'George',
  'transactions': [{'transaction-id': 22, 'amount': 2007},
   {'transaction-id': 488, 'amount': 1853},
   {'transaction-id': 507, 'amount': 1890},
   {'transaction-id': 508, 'amount': 1677},
   {'transaction-id': 933, 'amount': 1704},
   {'transaction-id': 979, 'amount': 1901},
   {'transaction-id': 1048, 'amount': 1792},
   {'transaction-id': 1193, 'amount': 1757},
   {'transaction-id': 1619, 'amount': 1613},
   {'transaction-id': 1646, 'amount': 1656},
   {'transaction-id': 1861, 'amount': 1821},
   {'transaction-id': 2000, 'amount': 1529},
   {'transaction-id': 2036, 'amount': 1754},
   {'transaction-id': 2289, 'amount': 1379},
   {'transaction-id': 2319, 'amount': 1642},
   {'transaction-id': 2678, 'amount': 1854},
   {'transaction-id': 3017, 'amount': 1763},
   {'transaction-id': 3101, 'amount': 1899},
   {'transaction-id': 3114, 'amount': 1866},
   {'transaction-id': 3234, 'amount': 1870},
   {'transaction-id': 3286, 'amount': 1724},
   {'transactio

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 15,
  'name': 'Alice',
  'transactions': [{'transaction-id': 428, 'amount': 194},
   {'transaction-id': 477, 'amount': 196},
   {'transaction-id': 535, 'amount': 200},
   {'transaction-id': 987, 'amount': 188},
   {'transaction-id': 1044, 'amount': 199},
   {'transaction-id': 1061, 'amount': 181},
   {'transaction-id': 1151, 'amount': 210},
   {'transaction-id': 1934, 'amount': 191},
   {'transaction-id': 2283, 'amount': 204},
   {'transaction-id': 2336, 'amount': 194},
   {'transaction-id': 2665, 'amount': 205},
   {'transaction-id': 3760, 'amount': 191},
   {'transaction-id': 4339, 'amount': 195},
   {'transaction-id': 4353, 'amount': 199},
   {'transaction-id': 4457, 'amount': 190},
   {'transaction-id': 4540, 'amount': 213},
   {'transaction-id': 4674, 'amount': 200},
   {'transaction-id': 4860, 'amount': 213},
   {'transaction-id': 5344, 'amount': 199},
   {'transaction-id': 5668, 'amount': 194},
   {'transaction-id': 5834, 'amount': 197},
   {'transaction-id': 5967, 'amou

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 43},
 {'name': 'Alice', 'count': 2},
 {'name': 'Alice', 'count': 3},
 {'name': 'Alice', 'count': 23},
 {'name': 'Alice', 'count': 13})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(43, 2, 3, 23, 13)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

25.49074074074074

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 428, 'amount': 194},
  {'transaction-id': 477, 'amount': 196},
  {'transaction-id': 535, 'amount': 200},
  {'transaction-id': 987, 'amount': 188},
  {'transaction-id': 1044, 'amount': 199},
  {'transaction-id': 1061, 'amount': 181},
  {'transaction-id': 1151, 'amount': 210},
  {'transaction-id': 1934, 'amount': 191},
  {'transaction-id': 2283, 'amount': 204},
  {'transaction-id': 2336, 'amount': 194},
  {'transaction-id': 2665, 'amount': 205},
  {'transaction-id': 3760, 'amount': 191},
  {'transaction-id': 4339, 'amount': 195},
  {'transaction-id': 4353, 'amount': 199},
  {'transaction-id': 4457, 'amount': 190},
  {'transaction-id': 4540, 'amount': 213},
  {'transaction-id': 4674, 'amount': 200},
  {'transaction-id': 4860, 'amount': 213},
  {'transaction-id': 5344, 'amount': 199},
  {'transaction-id': 5668, 'amount': 194},
  {'transaction-id': 5834, 'amount': 197},
  {'transaction-id': 5967, 'amount': 205},
  {'transaction-id': 6079, 'amount': 204},
  {'transaction

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 428, 'amount': 194},
 {'transaction-id': 477, 'amount': 196},
 {'transaction-id': 535, 'amount': 200})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(194, 196, 200)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

21.865056302215766

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 104), ('Alice', 104), ('Alice', 111), ('Alice', 113), ('Bob', 69), ('Bob', 71), ('Bob', 77), ('Bob', 78), ('Charlie', 108), ('Charlie', 108), ('Charlie', 115), ('Charlie', 117), ('Dan', 112), ('Dan', 113), ('Dan', 119), ('Dan', 121), ('Edith', 167), ('Edith', 168), ('Edith', 181), ('Edith', 182), ('Frank', 169), ('Frank', 169), ('Frank', 312), ('George', 155), ('George', 168), ('George', 322), ('Hannah', 131), ('Hannah', 131), ('Hannah', 141), ('Hannah', 142), ('Ingrid', 120), ('Ingrid', 120), ('Ingrid', 130), ('Ingrid', 130), ('Jerry', 120), ('Jerry', 120), ('Jerry', 130), ('Jerry', 130), ('Kevin', 72), ('Kevin', 72), ('Kevin', 78), ('Kevin', 78), ('Laura', 176), ('Laura', 177), ('Laura', 189), ('Laura', 189), ('Michael', 82), ('Michael', 83), ('Michael', 90), ('Michael', 90), ('Norbert', 107), ('Norbert', 108), ('Norbert', 116), ('Norbert', 117), ('Oliver', 132), ('Oliver', 132), ('Oliver', 143), ('Oliver', 143), ('Patricia', 132), ('Patricia', 132), ('Patricia', 143), ('P

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 432), ('Bob', 295), ('Charlie', 448), ('Dan', 465), ('Edith', 698), ('Frank', 650), ('George', 645), ('Hannah', 545), ('Ingrid', 500), ('Jerry', 500), ('Kevin', 300), ('Laura', 731), ('Michael', 345), ('Norbert', 448), ('Oliver', 550), ('Patricia', 550), ('Quinn', 350), ('Ray', 249), ('Sarah', 647), ('Tim', 641), ('Ursula', 644), ('Victor', 346), ('Wendy', 350), ('Xavier', 250), ('Yvonne', 491), ('Zelda', 201)]
CPU times: user 130 ms, sys: 11 ms, total: 141 ms
Wall time: 499 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,George,"[{'transaction-id': 22, 'amount': 2007}, {'tra..."
1,1,Oliver,"[{'transaction-id': 750, 'amount': 305}, {'tra..."
2,2,Oliver,"[{'transaction-id': 3022, 'amount': 457}, {'tr..."
3,3,Yvonne,"[{'transaction-id': 128, 'amount': 825}, {'tra..."
4,5,Xavier,"[{'transaction-id': 24, 'amount': 4281}, {'tra..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 156 ms, sys: 18.9 ms, total: 175 ms
Wall time: 1.32 s


name
Alice      432
Bob        295
Charlie    448
Dan        465
Edith      698
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'George', 'amount': 2007, 'transaction-id': 22},
 {'id': 0, 'name': 'George', 'amount': 1853, 'transaction-id': 488},
 {'id': 0, 'name': 'George', 'amount': 1890, 'transaction-id': 507})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,George,2007,22
1,0,George,1853,488
2,0,George,1890,507
3,0,George,1677,508
4,0,George,1704,933


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 167 ms, sys: 8.48 ms, total: 175 ms
Wall time: 1.09 s


name
Alice       11012
Bob         12215
Charlie     24925
Dan         21426
Edith       25683
Frank       19134
George      25757
Hannah      25829
Ingrid      21119
Jerry       16432
Kevin       11131
Laura       39549
Michael     18665
Norbert     10560
Oliver      20499
Patricia    25378
Quinn       17397
Ray          5431
Sarah       24409
Tim         38037
Ursula      14400
Victor      18164
Wendy       13919
Xavier      16470
Yvonne      15887
Zelda        6572
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()